In [124]:
import base64
import csv
import io
import json
import time
import requests
from nbt import nbt
from Core.settings import CSVDownloadsPath


def get_auction_bins() -> json:
    start_time = time.time()
    x = requests.get('https://api.hypixel.net/skyblock/auctions?key=41b2a888-f84c-4287-bf56-2012895c8e4d')
    parsed = json.dumps(x.json())
    json_loaded = json.loads(parsed)
    total_Pages = json_loaded['totalPages']
    page = 0
    auction_items = []
    while(page < total_Pages):
        x = requests.get('https://api.hypixel.net/skyblock/auctions?key=41b2a888-f84c-4287-bf56-2012895c8e4d&page=' + str(page))
        parsed = json.dumps(x.json())
        json_loaded = json.loads(parsed)
        for listing in json_loaded['auctions']:
            if('bin' in listing.keys()):
                if(listing['bin'] == True):
                    auction_items.append(listing)
        page=page+1

    end_time = time.time()
    print(end_time - start_time)
    return(auction_items)


def get_username(uuid):
    x = requests.get('https://playerdb.co/api/player/minecraft/' + str(uuid))
    parsed = json.dumps(x.json())
    json_loaded = json.loads(parsed)
    return json_loaded


def parse_auction_listings(auction_data):
    final_array = [['item_name', 'tier', 'category', 'starting_bid', 'count', 'auctioneer', 'start', 'end', 'bin', 'uuid','item_bytes']]
    for item in auction_data:
        passing_array = []
        passing_array.append(item['item_name'])
        passing_array.append(item['tier'])
        passing_array.append(item['category'])
        passing_array.append(item['starting_bid'])
        data = nbt.NBTFile(fileobj = io.BytesIO(base64.b64decode(item['item_bytes'])))
        count = data['i'][0]['Count']
        passing_array.append(count)
        passing_array.append(item['auctioneer'])
        passing_array.append(item['start'])
        passing_array.append(item['end'])
        passing_array.append(item['bin'])
        passing_array.append(item['uuid'])
        passing_array.append(item['item_bytes'])
        final_array.append(passing_array)
    print('Done')
    return final_array


def write_auctions_file(auctions_array):
    with open(CSVDownloadsPath + 'auction_data.csv', 'w', newline='\n', encoding='utf-8') as foo:
        writer = csv.writer(foo)
        writer.writerows(auctions_array)


def write_usernames_file(userid, username):
    with open(CSVDownloadsPath + 'username_dict.csv', 'a', newline='\n', encoding='utf-8') as foo:
        writer = csv.writer(foo)
        myarray = [userid, username]
        writer.writerow(myarray)


def read_auction_data():
    with open(CSVDownloadsPath + 'past_auctions_data.csv', encoding='utf-8') as csvDataFile:
        auction_dict = dict()
        csvReader = csv.reader(csvDataFile)
        print('file opened')
        for row in csvReader:
            uuid = row[9]
            if((uuid in auction_dict.keys()) != True):
                 auction_dict[uuid] = row
    return auction_dict

def read_username_dict():
    with open(CSVDownloadsPath + 'username_dict.csv') as csvDataFile:
        username_dict = dict()
        csvReader = csv.reader(csvDataFile)
        print('file opened')
        for row in csvReader:
            userid = row[0]
            if((userid in username_dict.keys()) != True):
                 username_dict[userid] = row[1]
    return username_dict





def isUsernameInDict(auctions_data):
    usernameDatabase = read_username_dict()
    for item in auctions_data:
        userid = item['auctioneer']
        if ((userid in usernameDatabase.keys()) != True):
            x = requests.get('https://playerdb.co/api/player/minecraft/' + userid)
            parsed = json.dumps(x.json())
            json_loaded = json.loads(parsed)
            username =json_loaded['data']['player']['username']
            usernameDatabase[userid] = username
            #print(username + ', ' + userid)
            write_usernames_file(userid, username)

In [129]:
auctions_data = get_auction_bins()
auctions_array= parse_auction_listings(auctions_data)
write_auctions_file(auctions_array)

29.804542064666748
Done


In [121]:
isUsernameInDict(auctions_data)

file opened


KeyboardInterrupt: 

In [114]:
def isAuctionInData(auctions_data):
    auction_read = read_auction_data()
    for item in auctions_data:
        uuid = item['uuid']
        final_array = []
        if ((uuid in auction_read.keys()) != True):
            passing_array = []
            passing_array.append(item['item_name'])
            passing_array.append(item['tier'])
            passing_array.append(item['category'])
            passing_array.append(item['starting_bid'])
            data = nbt.NBTFile(fileobj = io.BytesIO(base64.b64decode(item['item_bytes'])))
            count = data['i'][0]['Count']
            passing_array.append(count)
            passing_array.append(item['auctioneer'])
            passing_array.append(item['start'])
            passing_array.append(item['end'])
            passing_array.append(item['bin'])
            passing_array.append(item['uuid'])
            passing_array.append(item['item_bytes'])
            final_array.append(passing_array)
        write_past_auction_file(final_array)


def write_past_auction_file(passing_array):
    with open(CSVDownloadsPath + 'past_auctions_data.csv', 'a', newline='\n', encoding='utf-8') as foo:
        writer = csv.writer(foo)
        writer.writerows(passing_array)

In [130]:
isAuctionInData(auctions_data)

file opened
